In [183]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import datetime as dt
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

In [184]:
# Training data sets
benef = pd.read_csv('Train_Beneficiarydata.csv')
inpat = pd.read_csv('Train_Inpatientdata.csv')
outpat = pd.read_csv('Train_Outpatientdata.csv')
fraud = pd.read_csv('Train.csv')

In [185]:
print(benef.shape)
print(inpat.shape)
print(outpat.shape)
print(fraud.shape)

(138556, 25)
(40474, 30)
(517737, 27)
(5410, 2)


In [186]:
#Create columns for inpatient and outpatient data.
inpat['patientType'] = np.repeat('inpatient', len(inpat))
outpat['patientType'] = np.repeat('outpatient', len(outpat))

In [187]:
#Combining the inpatient and outpatient data
patient = pd.concat([inpat, outpat], axis=0)

In [188]:
# Combining patient dataset with beneficiary dataset 
full_data =pd.merge(patient, benef, on='BeneID', how='left')

## Full data has 558211 rows and 55 features

In [189]:
print(full_data.shape)

(558211, 55)


In [190]:
full_data1 = full_data.copy()

In [191]:
# Creating DaysAdmitted Feature by substracting claim start date from claim end date 
full_data1['ClaimStartDt'] = pd.to_datetime(full_data1['ClaimStartDt'])
full_data1['ClaimEndDt'] = pd.to_datetime(full_data1['ClaimEndDt'])
full_data1['DaysAdmitted'] = full_data1['ClaimEndDt'] - full_data1['ClaimStartDt']

In [192]:
# Chaging DaysAdmitted into integer object 
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(str)
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].str.replace('days', ' ')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted']. \
str.replace('00:00:00.000000000', '')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(int)

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'] + 1

C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


#### First Feature Engineered: DaysAdmittedMean `DaysAdmitted`

* Average duration of patients stay in hospital by provider

In [193]:
# Creating a separate dataframe of DaysAdmittedMean by Provider
days = pd.DataFrame(full_data1.groupby('Provider')['DaysAdmitted'].mean())
days = days.reset_index()

In [194]:
# Merging Potential Fraud dataset with DaysAdmittedMean dataset 
fraud1 = fraud.copy()
fraud2 = pd.merge(fraud1, days, on='Provider')

#### 2nd Feature: DeductibleAmountPaid Mean `DeductibleAmtPaid`

* Average deductible amount paid by patient per provider
* Fraudulent providers have higher average deductible amount paid by patient than non-fraudulent ones

In [195]:
DeductibleMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['DeductibleAmtPaid'].mean())

fraud2 = pd.merge(fraud2, DeductibleMean, on='Provider')

In [196]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid
0,PRV51001,No,2.440000,213.600000
1,PRV51003,Yes,4.674242,502.166667
2,PRV51004,No,2.429530,2.080537
3,PRV51005,Yes,2.088412,3.175966
4,PRV51007,No,1.958333,45.333333
...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857
5406,PRV57760,No,1.318182,0.000000
5407,PRV57761,No,2.390244,4.512195
5408,PRV57762,No,1.000000,0.000000


In [197]:
fraud2.groupby('PotentialFraud')['DeductibleAmtPaid'].mean()

PotentialFraud
No     135.498511
Yes    350.839395
Name: DeductibleAmtPaid, dtype: float64

#### 3rd Feature: Insurance Claim Amount Reimbursed Mean `InscClaimAmtReimbursed`

* Average insurance claim amount reimbursed for patient per provider

In [198]:
InscReimbursed = pd.DataFrame(full_data1.groupby('Provider') \
                              ['InscClaimAmtReimbursed'].mean())

fraud2 = pd.merge(fraud2, InscReimbursed, on='Provider')

In [199]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed
0,PRV51001,No,2.440000,213.600000,4185.600000
1,PRV51003,Yes,4.674242,502.166667,4588.409091
2,PRV51004,No,2.429530,2.080537,350.134228
3,PRV51005,Yes,2.088412,3.175966,241.124464
4,PRV51007,No,1.958333,45.333333,468.194444
...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000
5406,PRV57760,No,1.318182,0.000000,216.818182
5407,PRV57761,No,2.390244,4.512195,225.243902
5408,PRV57762,No,1.000000,0.000000,1900.000000


In [200]:
fraud2.groupby('PotentialFraud')['InscClaimAmtReimbursed'].mean()

PotentialFraud
No     1523.780804
Yes    3842.795104
Name: InscClaimAmtReimbursed, dtype: float64

#### 4th Feature: Insurance Covered Percentage Mean `InscCoveredPercent`

* Average insurance coverage=`InscClaimAmtReimbursed`/(`InscClaimAmtReimbursed`+`'DeductibleAmtPaid'`)

In [201]:
full_data1['InscCoveredPercent'] = full_data1['InscClaimAmtReimbursed'] \
/(full_data1['InscClaimAmtReimbursed'] + full_data1['DeductibleAmtPaid'])

InscCoveredPercent = pd.DataFrame(full_data1.groupby('Provider') \
                                  ['InscCoveredPercent'].mean())

fraud2 = pd.merge(fraud2, InscCoveredPercent, on='Provider')

In [202]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303
...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000


In [203]:
fraud2.groupby('PotentialFraud')['InscCoveredPercent'].mean()

PotentialFraud
No     0.962077
Yes    0.934837
Name: InscCoveredPercent, dtype: float64

#### 5th Feature: Total Claim Amount Mean `TotalClaimAmount`

* Total Claim Amount = `InscClaimAmtReimbursed`+`DeductibleAmtPaid`

In [204]:
full_data1['TotalClaimAmount'] = full_data1['InscClaimAmtReimbursed']\
+ full_data1['DeductibleAmtPaid']

TotalClaimAmount = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalClaimAmount'].mean())

fraud2 = pd.merge(fraud2, TotalClaimAmount, on='Provider')

In [205]:
fraud2.groupby('PotentialFraud')['TotalClaimAmount'].mean()

PotentialFraud
No     1645.778736
Yes    4142.438122
Name: TotalClaimAmount, dtype: float64

#### 6th Feature: Daily Total Charge `DailyTotalCharge`

* Total Daily Charge = `TotalClaimAmount`/`DaysAdmitted`

In [206]:
full_data1['DailyTotalCharge'] = full_data1['TotalClaimAmount'] \
/full_data1['DaysAdmitted']

DailyTotalCharge = pd.DataFrame(full_data1.groupby('Provider') \
                                ['DailyTotalCharge'].mean())

fraud2 = pd.merge(fraud2, DailyTotalCharge, on='Provider')

In [207]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944
...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000


In [208]:
fraud2.groupby('PotentialFraud')['DailyTotalCharge'].mean()

PotentialFraud
No     471.273799
Yes    952.785225
Name: DailyTotalCharge, dtype: float64

#### Features 7-10: Annual Reimbursement and Deductible Paid Amounts for Inpatient and Outpatient
#### `IPAnnualReimbursementAmt`, `IPAnnualDeductibleAmt`, `OPAnnualReimbursementAmt`, `	OPAnnualDeductibleAmt`

In [209]:
IPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['IPAnnualReimbursementAmt'].mean())

IPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['IPAnnualDeductibleAmt'].mean())

OPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['OPAnnualReimbursementAmt'].mean())

OPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['OPAnnualDeductibleAmt'].mean())

In [210]:
fraud2 = pd.merge(fraud2, IPAnnReimbMean, on='Provider')
fraud2 = pd.merge(fraud2, IPAnnDeductMean, on='Provider')
fraud2 = pd.merge(fraud2, OPAnnReimbMean, on='Provider')
fraud2 = pd.merge(fraud2, OPAnnDeductMean, on='Provider')

In [211]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222
...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000


In [212]:
fraud2.groupby('PotentialFraud')['IPAnnualReimbursementAmt'].mean()

PotentialFraud
No     5868.975601
Yes    9052.076164
Name: IPAnnualReimbursementAmt, dtype: float64

In [213]:
fraud2.groupby('PotentialFraud')['IPAnnualDeductibleAmt'].mean()

PotentialFraud
No     638.107001
Yes    946.817676
Name: IPAnnualDeductibleAmt, dtype: float64

In [214]:
fraud2.groupby('PotentialFraud')['OPAnnualReimbursementAmt'].mean()

PotentialFraud
No     2191.723203
Yes    2273.113210
Name: OPAnnualReimbursementAmt, dtype: float64

In [215]:
fraud2.groupby('PotentialFraud')['OPAnnualDeductibleAmt'].mean()

PotentialFraud
No     628.840769
Yes    631.565539
Name: OPAnnualDeductibleAmt, dtype: float64

#### Feature 11: Total Annual Claim Amount Mean `TotalAnnClaimAmt`

* Total Annual Claim Amount=`IPAnnualReimbursementAmt`+`IPAnnualDeductibleAmt`+`OPAnnualReimbursementAmt`+`OPAnnualDeductibleAm`

In [216]:
full_data1['TotalAnnClaimAmt'] = full_data1['IPAnnualReimbursementAmt']+ \
full_data1['IPAnnualDeductibleAmt']+full_data1['OPAnnualReimbursementAmt'] \
+ full_data1['OPAnnualDeductibleAmt']

TotalAnnClaimAmt = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalAnnClaimAmt'].mean())

fraud2 = pd.merge(fraud2, TotalAnnClaimAmt, on='Provider')

In [217]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000


In [218]:
fraud2.groupby('PotentialFraud')['TotalAnnClaimAmt'].mean()

PotentialFraud
No      9327.646575
Yes    12903.572589
Name: TotalAnnClaimAmt, dtype: float64

#### Add number of doctors `NumOfDoctors`

In [219]:
doctors= full_data1.groupby(['Provider', 'AttendingPhysician']) \
['AttendingPhysician'].count().reset_index(name='NumOfDoctors'). \
groupby('Provider')['NumOfDoctors'].count().reset_index()

fraud2 = pd.merge(fraud2, doctors, on='Provider')

In [220]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1


#### Add number of patients `NumOfPatients`

In [221]:
patient= full_data.groupby(['Provider','BeneID'])['BeneID'].count(). \
reset_index(name='NumOfPatients').groupby('Provider')['NumOfPatients']. \
count().reset_index()

fraud2 = pd.merge(fraud2, patient, on='Provider')

In [222]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1


#### Add Type of Service  `ServiceType`

In [223]:
a = full_data1.groupby(['Provider', 'patientType'])['BeneID'].count().reset_index(name='count').\
drop('count', axis=1)

a_list= list(zip(a.Provider, a.patientType))

providerDict= {}

for ele in a_list:
    if ele[0] not in providerDict:
        providerDict[ele[0]]= ele[1]    
    else:
        providerDict[ele[0]] = 'both'
        

providerService = pd.DataFrame(providerDict.keys(), providerDict.values()).reset_index().\
rename(columns={'index':'ServiceType', 0:'Provider'})

In [224]:
fraud2 = pd.merge(fraud2, providerService, on='Provider')
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient


#### Add number of claims `NumOfClms`
* Number of claims filed per provider

In [225]:
claims= full_data1.groupby(['Provider', 'ClaimID'])['ClaimID'].count().\
reset_index(name='NumOfClms').groupby('Provider')['NumOfClms'].count().reset_index()

fraud2 = pd.merge(fraud2, claims, on='Provider')

In [226]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1


In [227]:
fraud5 = fraud2.copy()

#### Add weekly claims `WeeklyClaims`

In [228]:
full_data1['startClaimWeek']= full_data1['ClaimStartDt'].dt.week

C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [229]:
weeklyClaims= full_data1.groupby(['Provider','startClaimWeek', 'ClaimID'])['ClaimID'].count().reset_index(name='WeeklyClaims').\
groupby(['Provider'])['WeeklyClaims'].count().reset_index()

In [230]:
fraud5= pd.merge(fraud5, weeklyClaims, on='Provider', how='left')

In [231]:
fraud5

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1


#### Add average duration of treatment per claims

#### Add monthly claims `MonthlyClaims`

In [232]:
full_data1['startClaimMonth'] = full_data1['ClaimStartDt'].dt.month

monthly = full_data1.groupby(['Provider','startClaimMonth', 'ClaimID'])['ClaimID'].count().reset_index(name='MonthlyClaims').\
groupby(['Provider'])['MonthlyClaims'].count().reset_index()

In [233]:
fraud5 = pd.merge(fraud5, monthly, on='Provider', how='left')

In [234]:
fraud5

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1


#### Add number of states each provider operates in `numState`

In [235]:
numStates = full_data1.groupby(['Provider', 'State'])['State'].count().reset_index(name='a').groupby('Provider')['State'].count().\
reset_index(name='numStates')

In [236]:
fraud5 = pd.merge(fraud5, numStates, on='Provider')

In [237]:
fraud5

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1


#### Add number of counties each provider operates in `numCounties`

In [238]:
numCounties = full_data.groupby(['Provider', 'County'])['County'].count().reset_index(name='a'). \
groupby('Provider')['County'].count().reset_index(name='numCounties')

In [239]:
fraud5 = pd.merge(fraud5, numCounties, on='Provider')

In [240]:
fraud5

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1


In [241]:
fraud6 = fraud5.copy()

#### Add average number of chronic conditions per provider `NumChronicCond`
#### Rename `NumChronicCond` to `AvgChronic`, rename `NumClaims` to `AvgClaim`

In [242]:
patientChronic = full_data1.filter(regex='Chronic').replace(to_replace=2, value=0).sum(axis=1).\
reset_index(name='NumChronicCond')

In [243]:
chronic = pd.concat([patientChronic, full_data1[['Provider', 'ClaimID']]], axis=1)

In [244]:
avgChronic = chronic.groupby(['Provider', 'NumChronicCond'])['ClaimID'].count().reset_index(name='NumClaims').\
groupby('Provider').agg({'NumChronicCond':'mean', 'NumClaims':'mean'}).reset_index().\
rename(columns={'NumChronicCond':'AvgChronic', 'NumClaims':'AvgClaim'})

In [245]:
fraud6 = pd.merge(fraud6, avgChronic, on='Provider')

In [246]:
fraud6

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000


#### Add number of claim admit diagnosis codes per provider `numDiffDiagnosisCode`

In [247]:
numDiffDiagnosisCode= full_data1.groupby(['Provider', 'ClmAdmitDiagnosisCode'])['ClmAdmitDiagnosisCode'].count(). \
reset_index(name='a').groupby('Provider')['ClmAdmitDiagnosisCode'].count().reset_index(name='numDiffDiagnosisCode')

In [248]:
fraud6 = pd.merge(fraud6, numDiffDiagnosisCode, on='Provider', how='left')

In [249]:
fraud6

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0


#### Add number of group diagnosis codes per provider `numDiffGroupDiagCode`

In [250]:
numDiffGroupDiagCode = full_data1.groupby(['Provider', 'DiagnosisGroupCode'])['DiagnosisGroupCode']. \
count().reset_index(name='a').groupby('Provider')['DiagnosisGroupCode']. \
count().reset_index(name='numDiffGroupDiagCode')

In [251]:
fraud6 = pd.merge(fraud6, numDiffGroupDiagCode, on='Provider', how='left')

In [252]:
fraud6

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN


In [253]:
fraud7 = fraud6.copy()

#### Adding average of patients' age per provider `AvgAgeWhenServed`

In [254]:
# Calculating the age of the patient at the time of their service
full_data1['ClaimStartDt2'] =  pd.to_datetime(full_data1['ClaimStartDt'], format='%Y/%m/%d')
full_data1['DOB'] =  pd.to_datetime(full_data1['DOB'], format='%Y/%m/%d')
full_data1['AgeWhenServed'] = full_data1['ClaimStartDt'] - full_data1['DOB']

In [255]:
# Converting the unit of AgeWhenServed to year from days
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'] / np.timedelta64(1, 'Y')

In [256]:
# Converting AgeWhenServed to int from float
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].fillna(-1)
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].astype(int)
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].replace(-1, np.nan)

In [257]:
avgage = full_data1.groupby(['Provider', 'BeneID'])['AgeWhenServed']. \
mean().reset_index(name = "AvgAgeWhenServed").dropna() \
.groupby('Provider')['AvgAgeWhenServed'].mean().reset_index()

In [258]:
fraud7 = pd.merge(fraud7, avgage, on='Provider', how='left')

In [259]:
fraud7

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000


#### Adding the count of Duplicated Patient ID per Provider `NumOfDuplicatedBeneID`

In [260]:
inpatient = full_data1.loc[full_data1.patientType == 'inpatient']
outpatient = full_data1.loc[full_data1.patientType == 'outpatient']

In [261]:
# Dataframe of duplicated data for inpatients
inpatient_d = inpatient[inpatient.duplicated(['ClmAdmitDiagnosisCode', \
                                              'DiagnosisGroupCode', \
                                              'ClmDiagnosisCode_1', \
                                              'ClmDiagnosisCode_2', \
                                              'ClmDiagnosisCode_3', \
                                              'ClmDiagnosisCode_4', \
                                              'ClmDiagnosisCode_5', \
                                              'ClmDiagnosisCode_6', \
                                              'ClmDiagnosisCode_7', \
                                              'ClmDiagnosisCode_8', \
                                              'ClmDiagnosisCode_9', \
                                              'ClmDiagnosisCode_10', \
                                              'ClmProcedureCode_1', \
                                              'ClmProcedureCode_2', \
                                              'ClmProcedureCode_3', \
                                              'ClmProcedureCode_4', \
                                              'ClmProcedureCode_5', \
                                              'ClmProcedureCode_6'], \
                                             keep = False)]

In [262]:
# Dataframe of duplicated data for inpatients
outpatient_d = outpatient[outpatient.duplicated(['ClmAdmitDiagnosisCode', \
                                              'DiagnosisGroupCode', \
                                              'ClmDiagnosisCode_1', \
                                              'ClmDiagnosisCode_2', \
                                              'ClmDiagnosisCode_3', \
                                              'ClmDiagnosisCode_4', \
                                              'ClmDiagnosisCode_5', \
                                              'ClmDiagnosisCode_6', \
                                              'ClmDiagnosisCode_7', \
                                              'ClmDiagnosisCode_8', \
                                              'ClmDiagnosisCode_9', \
                                              'ClmDiagnosisCode_10', \
                                              'ClmProcedureCode_1', \
                                              'ClmProcedureCode_2', \
                                              'ClmProcedureCode_3', \
                                              'ClmProcedureCode_4', \
                                              'ClmProcedureCode_5', \
                                              'ClmProcedureCode_6'], \
                                             keep = False)]

In [263]:
# totalpatient_d = Duplicated inpatient + Duplicated outpatient
totalpatient_d = pd.concat([inpatient_d, outpatient_d], axis=0)

In [264]:
duplicatedBeneID = totalpatient_d.groupby('Provider')['BeneID'].value_counts().reset_index(name = "ValueCount")
duplicatedBeneID = duplicatedBeneID.loc[duplicatedBeneID['ValueCount'] > 1]
duplicatedBeneID = duplicatedBeneID.groupby('Provider')['BeneID'].count().reset_index(name = "NumOfDuplicatedBeneID")

In [265]:
fraud7 = pd.merge(fraud7, duplicatedBeneID, on='Provider', how='left')

In [266]:
fraud7['NumOfDuplicatedBeneID'].fillna(0, inplace=True)

In [268]:
fraud7

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802,2.0
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649,113.0
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667,3.0
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667,3.0
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995,3.0
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000,0.0


#### Adding count of duplicated attending physician per provider `NumOfDuplicatedAttendingPhysician` 

In [269]:
duplicatedPhysician = totalpatient_d.groupby('Provider')['AttendingPhysician'].value_counts().reset_index(name = "ValueCount")
duplicatedPhysician = duplicatedPhysician.loc[duplicatedPhysician['ValueCount'] > 1]
duplicatedPhysician = duplicatedPhysician.groupby('Provider')['AttendingPhysician'].count().reset_index(name = "NumOfDuplicatedAttendingPhysician")

In [270]:
fraud7 = pd.merge(fraud7, duplicatedPhysician, on='Provider', how='left')
fraud7['NumOfDuplicatedAttendingPhysician'].fillna(0, inplace=True)

In [272]:
fraud7

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802,2.0,16.0
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649,113.0,6.0
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667,3.0,1.0
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667,3.0,1.0
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995,3.0,2.0
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000,0.0,0.0


#### Adding count of duplicated claims per provider `NumOfDuplicatedClaims`

In [273]:
duplicatedClaims = totalpatient_d.groupby('Provider').agg({'ClaimID' : 'count'}).reset_index()
duplicatedClaims = duplicatedClaims.rename({'ClaimID' : 'NumOfDuplicatedClaims'}, axis=1)

In [274]:
fraud7 = pd.merge(fraud7, duplicatedClaims, on='Provider', how='left')
fraud7['NumOfDuplicatedClaims'].fillna(0, inplace=True)

In [276]:
fraud7

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802,2.0,16.0,72.0
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649,113.0,6.0,530.0
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667,3.0,1.0,15.0
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667,3.0,1.0,13.0
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995,3.0,2.0,32.0
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000,0.0,0.0,1.0


#### Adding the count of patients with different number of chronic conditions `Chronic0`-`Chronic12`

In [277]:
fraud8 = fraud7.copy()

In [278]:
full_data2 = full_data1.copy()

In [279]:
full_data2 = full_data2.replace({'RenalDiseaseIndicator' : 'Y'}, 1)

full_data2['RenalDiseaseIndicator'] = full_data2['RenalDiseaseIndicator'].astype(int)

In [280]:
full_data2 = full_data2.replace({'ChronicCond_Alzheimer' : 2,'ChronicCond_Heartfailure' : 2,'ChronicCond_KidneyDisease' : 2,'ChronicCond_Cancer' : 2,'ChronicCond_ObstrPulmonary' : 2,'ChronicCond_Depression' : 2,'ChronicCond_Diabetes' : 2,'ChronicCond_IschemicHeart' : 2,'ChronicCond_Osteoporasis' : 2,'ChronicCond_rheumatoidarthritis' : 2,'ChronicCond_stroke' : 2}, 0)

In [281]:
full_data2['NumChronicCond'] = full_data2['ChronicCond_Alzheimer']+full_data2['ChronicCond_Heartfailure']+full_data2['ChronicCond_KidneyDisease']+full_data2['ChronicCond_Cancer']+full_data2['ChronicCond_ObstrPulmonary']+full_data2['ChronicCond_Depression']+full_data2['ChronicCond_Diabetes']+full_data2['ChronicCond_IschemicHeart']+full_data2['ChronicCond_Osteoporasis']+full_data2['ChronicCond_rheumatoidarthritis']+full_data2['ChronicCond_stroke']+full_data2['RenalDiseaseIndicator']

In [282]:
chronicConditions = full_data2.groupby(['Provider', 'BeneID'])['NumChronicCond'].value_counts().reset_index(name = 'NumChronicCondCount')
chronicConditions['NumChronicCondCount'] = chronicConditions['NumChronicCondCount'].apply(lambda x: 1 if x >= 1 else 0)
chronicConditions = chronicConditions.groupby('Provider')['NumChronicCond'].value_counts().reset_index(name = 'NumChronicCondCount')

In [283]:
chronicConditions = chronicConditions.groupby(['Provider', 'NumChronicCond']).NumChronicCondCount.sum()
chronicConditions = chronicConditions.unstack(level='NumChronicCond').reset_index()
chronicConditions = chronicConditions.rename({0 : 'ChronicCond0', 
                                              1 : 'ChronicCond1', 
                                              2 : 'ChronicCond2',
                                              3 : 'ChronicCond3',
                                              4 : 'ChronicCond4',
                                              5 : 'ChronicCond5',
                                              6 : 'ChronicCond6',
                                              7 : 'ChronicCond7',
                                              8 : 'ChronicCond8',
                                              9 : 'ChronicCond9',
                                              10 : 'ChronicCond10',
                                              11 : 'ChronicCond11',
                                              12 : 'ChronicCond12'}, axis=1)
chronicConditions.fillna(0, inplace=True)
chronicConditions

NumChronicCond,Provider,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12
0,PRV51001,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0
1,PRV51003,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0
2,PRV51004,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0
3,PRV51005,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0
4,PRV51007,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0
5406,PRV57760,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5407,PRV57761,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0
5408,PRV57762,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [284]:
fraud8 = pd.merge(fraud8, chronicConditions, on = 'Provider', how = 'left')

In [286]:
fraud8

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Adding counts of each gender of patients per provider `Gender1`, `Gender2`

In [287]:
gender = full_data1.groupby(['Provider', 'BeneID', 'Gender'])['Gender'].count().reset_index(name = 'GenderCount')
gender['GenderCount'] = gender['GenderCount'].apply(lambda x: 1 if x >= 1 else 0)
gender = gender.groupby('Provider')['Gender'].value_counts().reset_index(name = 'GenderCount')

In [288]:
gender['Gender1'] = 0
gender['Gender2'] = 0

for index in gender.index:
    if gender['Gender'][index] == 1:
        gender['Gender1'][index] += gender['GenderCount'][index]
    if gender['Gender'][index] == 2:
        gender['Gender2'][index] += gender['GenderCount'][index]

C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [289]:
gender = gender.drop('Gender', axis = 1)
gender = gender.drop('GenderCount', axis = 1)

In [290]:
genderCount = gender.groupby('Provider').agg({'Gender1' : 'sum', 'Gender2' : 'sum'}).reset_index()

In [291]:
fraud8 = pd.merge(fraud8, genderCount, on = 'Provider', how = 'left')

In [292]:
fraud8

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


#### Adding counts of each race of patients per provider `Race1`, `Race2`, `Race3`, `Race5`

In [293]:
race = full_data1.groupby(['Provider', 'BeneID', 'Race'])['Race'].count().reset_index(name = 'RaceCount')
race['RaceCount'] = race['RaceCount'].apply(lambda x: 1 if x >= 1 else 0)
race = race.groupby('Provider')['Race'].value_counts().reset_index(name = 'RaceCount')

In [294]:
race['Race1'] = 0
race['Race2'] = 0
race['Race3'] = 0
race['Race5'] = 0

for index in race.index:
    if race['Race'][index] == 1:
        race['Race1'][index] += race['RaceCount'][index]
    if race['Race'][index] == 2:
        race['Race2'][index] += race['RaceCount'][index]
    if race['Race'][index] == 3:
        race['Race3'][index] += race['RaceCount'][index]
    if race['Race'][index] == 5:
        race['Race5'][index] += race['RaceCount'][index]

C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\李岳\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [295]:
race = race.drop('Race', axis = 1)
race = race.drop('RaceCount', axis = 1)

In [296]:
raceCount = race.groupby('Provider').agg({'Race1' : 'sum', 'Race2' : 'sum', 'Race3' : 'sum', 'Race5' : 'sum'}).reset_index()

In [297]:
fraud8 = pd.merge(fraud8, raceCount, on = 'Provider', how = 'left')

In [298]:
fraud8

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,WeeklyClaims,MonthlyClaims,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000,14,24,both,25,25,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091,44,117,both,132,132,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221,38,138,outpatient,149,149,149,9,28,4.500000,14.900000,25.0,NaN,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309,6,495,outpatient,1165,1165,1165,4,26,4.500000,116.500000,158.0,NaN,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444,10,58,both,72,72,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571,1,24,outpatient,28,28,28,1,2,5.500000,2.800000,3.0,NaN,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455,3,9,outpatient,22,22,22,1,3,4.600000,4.400000,7.0,NaN,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634,2,67,outpatient,82,82,82,1,2,4.500000,8.200000,16.0,NaN,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000,1,1,outpatient,1,1,1,1,1,5.000000,1.000000,1.0,NaN,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0


#### Replace na values with 0

In [299]:
fraud9 = fraud8.copy()

In [300]:
fraud9 = fraud9.replace(np.nan,0)

### Save Dataset

In [301]:
fraud9.to_csv('train_dataset.csv')

## Feature Engineering 2
### - After Performing Random Trees Feature Selection
* Remove 8 features, 40 features remaining  

In [302]:
train_data = fraud9.copy()

In [303]:
#dropping columns after performing Random Trees feature importance 

train_data.drop(['WeeklyClaims', 'MonthlyClaims', 'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
                'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'DeductibleAmtPaid',
                'InscClaimAmtReimbursed'], axis=1, inplace=True)

In [304]:
train_data

,Provider,PotentialFraud,DaysAdmitted,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5
0,PRV51001,No,2.440000,0.975656,4399.200000,1061.072000,21582.240000,14,24,both,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0
1,PRV51003,Yes,4.674242,0.912167,5090.575758,1298.615410,11914.909091,44,117,both,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1
2,PRV51004,No,2.429530,0.978485,352.214765,250.363050,7604.483221,38,138,outpatient,149,9,28,4.500000,14.900000,25.0,0.0,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0
3,PRV51005,Yes,2.088412,0.980747,244.300429,196.533055,6749.216309,6,495,outpatient,1165,4,26,4.500000,116.500000,158.0,0.0,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0
4,PRV51007,No,1.958333,0.985303,513.527778,243.681944,5694.444444,10,58,both,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,0.983401,384.642857,200.629252,8548.428571,1,24,outpatient,28,1,2,5.500000,2.800000,3.0,0.0,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4
5406,PRV57760,No,1.318182,1.000000,216.818182,216.022727,5520.545455,3,9,outpatient,22,1,3,4.600000,4.400000,7.0,0.0,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0
5407,PRV57761,No,2.390244,0.935979,229.756098,157.134674,11248.414634,2,67,outpatient,82,1,2,4.500000,8.200000,16.0,0.0,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3
5408,PRV57762,No,1.000000,1.000000,1900.000000,1900.000000,19008.000000,1,1,outpatient,1,1,1,5.000000,1.000000,1.0,0.0,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0


#### New Feature: Number of Distinct Operating Physicians per provider `NumDistincOpPhy`

In [305]:
opPhy = full_data1.groupby(['Provider', 'OperatingPhysician'])['OperatingPhysician'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincOpPhy')

In [306]:
train_data = pd.merge(train_data, opPhy, on = 'Provider', how = 'left')

In [307]:
train_data

,Provider,PotentialFraud,DaysAdmitted,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5,NumDistincOpPhy
0,PRV51001,No,2.440000,0.975656,4399.200000,1061.072000,21582.240000,14,24,both,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0,5.0
1,PRV51003,Yes,4.674242,0.912167,5090.575758,1298.615410,11914.909091,44,117,both,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1,7.0
2,PRV51004,No,2.429530,0.978485,352.214765,250.363050,7604.483221,38,138,outpatient,149,9,28,4.500000,14.900000,25.0,0.0,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0,19.0
3,PRV51005,Yes,2.088412,0.980747,244.300429,196.533055,6749.216309,6,495,outpatient,1165,4,26,4.500000,116.500000,158.0,0.0,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0,6.0
4,PRV51007,No,1.958333,0.985303,513.527778,243.681944,5694.444444,10,58,both,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,0.983401,384.642857,200.629252,8548.428571,1,24,outpatient,28,1,2,5.500000,2.800000,3.0,0.0,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4,1.0
5406,PRV57760,No,1.318182,1.000000,216.818182,216.022727,5520.545455,3,9,outpatient,22,1,3,4.600000,4.400000,7.0,0.0,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0,2.0
5407,PRV57761,No,2.390244,0.935979,229.756098,157.134674,11248.414634,2,67,outpatient,82,1,2,4.500000,8.200000,16.0,0.0,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3,5.0
5408,PRV57762,No,1.000000,1.000000,1900.000000,1900.000000,19008.000000,1,1,outpatient,1,1,1,5.000000,1.000000,1.0,0.0,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,NaN


#### Number of Distinct "Other Physicians" per provider `NumDistincOtherPhy`

In [308]:
otherPhy = full_data1.groupby(['Provider', 'OtherPhysician'])['OtherPhysician'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincOtherPhy')

In [309]:
train_data = pd.merge(train_data, otherPhy, on = 'Provider', how = 'left')

In [310]:
train_data

,Provider,PotentialFraud,DaysAdmitted,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5,NumDistincOpPhy,NumDistincOtherPhy
0,PRV51001,No,2.440000,0.975656,4399.200000,1061.072000,21582.240000,14,24,both,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0,5.0,8.0
1,PRV51003,Yes,4.674242,0.912167,5090.575758,1298.615410,11914.909091,44,117,both,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1,7.0,22.0
2,PRV51004,No,2.429530,0.978485,352.214765,250.363050,7604.483221,38,138,outpatient,149,9,28,4.500000,14.900000,25.0,0.0,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0,19.0,26.0
3,PRV51005,Yes,2.088412,0.980747,244.300429,196.533055,6749.216309,6,495,outpatient,1165,4,26,4.500000,116.500000,158.0,0.0,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0,6.0,4.0
4,PRV51007,No,1.958333,0.985303,513.527778,243.681944,5694.444444,10,58,both,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0,5.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,0.983401,384.642857,200.629252,8548.428571,1,24,outpatient,28,1,2,5.500000,2.800000,3.0,0.0,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4,1.0,1.0
5406,PRV57760,No,1.318182,1.000000,216.818182,216.022727,5520.545455,3,9,outpatient,22,1,3,4.600000,4.400000,7.0,0.0,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0,2.0,2.0
5407,PRV57761,No,2.390244,0.935979,229.756098,157.134674,11248.414634,2,67,outpatient,82,1,2,4.500000,8.200000,16.0,0.0,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3,5.0,2.0
5408,PRV57762,No,1.000000,1.000000,1900.000000,1900.000000,19008.000000,1,1,outpatient,1,1,1,5.000000,1.000000,1.0,0.0,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,NaN,NaN


#### Number of Distinct Claim Diagnosis Codes per provider `NumDistincClmDiagCode1`-`NumDistincClmDiagCode10`

In [311]:
diagCode1 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_1'])['ClmDiagnosisCode_1'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode1')

diagCode2 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_2'])['ClmDiagnosisCode_2'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode2')

diagCode3 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_3'])['ClmDiagnosisCode_3'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode3')

diagCode4 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_4'])['ClmDiagnosisCode_4'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode4')

diagCode5 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_5'])['ClmDiagnosisCode_5'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode5')

diagCode6= full_data1.groupby(['Provider', 'ClmDiagnosisCode_6'])['ClmDiagnosisCode_6'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode6')

diagCode7 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_7'])['ClmDiagnosisCode_7'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode7')

diagCode8 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_8'])['ClmDiagnosisCode_8'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode8')

diagCode9 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_9'])['ClmDiagnosisCode_9'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode9')

diagCode10 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_10'])['ClmDiagnosisCode_10'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode10')

In [312]:
train_data = pd.merge(train_data, diagCode1, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode2, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode3, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode4, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode5, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode6, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode7, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode8, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode9, on='Provider', how='left')
train_data = pd.merge(train_data, diagCode10, on='Provider', how='left')

In [313]:
train_data

,Provider,PotentialFraud,DaysAdmitted,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5,NumDistincOpPhy,NumDistincOtherPhy,NumDistincClmDiagCode1,NumDistincClmDiagCode2,NumDistincClmDiagCode3,NumDistincClmDiagCode4,NumDistincClmDiagCode5,NumDistincClmDiagCode6,NumDistincClmDiagCode7,NumDistincClmDiagCode8,NumDistincClmDiagCode9,NumDistincClmDiagCode10
0,PRV51001,No,2.440000,0.975656,4399.200000,1061.072000,21582.240000,14,24,both,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0,5.0,8.0,23.0,15.0,10.0,8.0,8.0,8.0,3.0,2.0,1.0,NaN
1,PRV51003,Yes,4.674242,0.912167,5090.575758,1298.615410,11914.909091,44,117,both,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1,7.0,22.0,115.0,86.0,65.0,59.0,55.0,47.0,47.0,42.0,38.0,5.0
2,PRV51004,No,2.429530,0.978485,352.214765,250.363050,7604.483221,38,138,outpatient,149,9,28,4.500000,14.900000,25.0,0.0,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0,19.0,26.0,128.0,68.0,52.0,32.0,21.0,12.0,8.0,7.0,4.0,NaN
3,PRV51005,Yes,2.088412,0.980747,244.300429,196.533055,6749.216309,6,495,outpatient,1165,4,26,4.500000,116.500000,158.0,0.0,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0,6.0,4.0,809.0,327.0,232.0,186.0,113.0,79.0,61.0,40.0,25.0,1.0
4,PRV51007,No,1.958333,0.985303,513.527778,243.681944,5694.444444,10,58,both,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0,5.0,7.0,71.0,32.0,28.0,21.0,14.0,12.0,8.0,6.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,0.983401,384.642857,200.629252,8548.428571,1,24,outpatient,28,1,2,5.500000,2.800000,3.0,0.0,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4,1.0,1.0,26.0,11.0,8.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN
5406,PRV57760,No,1.318182,1.000000,216.818182,216.022727,5520.545455,3,9,outpatient,22,1,3,4.600000,4.400000,7.0,0.0,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0,2.0,2.0,20.0,10.0,8.0,5.0,3.0,2.0,1.0,NaN,NaN,NaN
5407,PRV57761,No,2.390244,0.935979,229.756098,157.134674,11248.414634,2,67,outpatient,82,1,2,4.500000,8.200000,16.0,0.0,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3,5.0,2.0,81.0,47.0,27.0,23.0,11.0,8.0,4.0,2.0,2.0,NaN
5408,PRV57762,No,1.000000,1.000000,1900.000000,1900.000000,19008.000000,1,1,outpatient,1,1,1,5.000000,1.000000,1.0,0.0,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Number of Distinct Procedure Diagnosis Codes per provider `NumDistincClmProCode1`-`NumDistincClmProCode5`

In [314]:
proCode1= full_data1.groupby(['Provider', 'ClmProcedureCode_1'])['ClmProcedureCode_1'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode1')


proCode2 = full_data1.groupby(['Provider', 'ClmProcedureCode_2'])['ClmProcedureCode_2'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode2')


proCode3 = full_data1.groupby(['Provider', 'ClmProcedureCode_3'])['ClmProcedureCode_3'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode3')

proCode4 = full_data1.groupby(['Provider', 'ClmProcedureCode_4'])['ClmProcedureCode_4'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode4')


proCode5 = full_data1.groupby(['Provider', 'ClmProcedureCode_5'])['ClmProcedureCode_5'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode5')

In [315]:
train_data = pd.merge(train_data, proCode1, on='Provider', how='left')
train_data = pd.merge(train_data, proCode2, on='Provider', how='left')
train_data = pd.merge(train_data, proCode3, on='Provider', how='left')
train_data = pd.merge(train_data, proCode4, on='Provider', how='left')
train_data = pd.merge(train_data, proCode5, on='Provider', how='left')

In [316]:
train_data

,Provider,PotentialFraud,DaysAdmitted,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,numStates,numCounties,AvgChronic,AvgClaim,numDiffDiagnosisCode,numDiffGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5,NumDistincOpPhy,NumDistincOtherPhy,NumDistincClmDiagCode1,NumDistincClmDiagCode2,NumDistincClmDiagCode3,NumDistincClmDiagCode4,NumDistincClmDiagCode5,NumDistincClmDiagCode6,NumDistincClmDiagCode7,NumDistincClmDiagCode8,NumDistincClmDiagCode9,NumDistincClmDiagCode10,NumDistincClmProCode1,NumDistincClmProCode2,NumDistincClmProCode3,NumDistincClmProCode4,NumDistincClmProCode5
0,PRV51001,No,2.440000,0.975656,4399.200000,1061.072000,21582.240000,14,24,both,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0,5.0,8.0,23.0,15.0,10.0,8.0,8.0,8.0,3.0,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
1,PRV51003,Yes,4.674242,0.912167,5090.575758,1298.615410,11914.909091,44,117,both,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1,7.0,22.0,115.0,86.0,65.0,59.0,55.0,47.0,47.0,42.0,38.0,5.0,32.0,7.0,NaN,NaN,NaN
2,PRV51004,No,2.429530,0.978485,352.214765,250.363050,7604.483221,38,138,outpatient,149,9,28,4.500000,14.900000,25.0,0.0,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0,19.0,26.0,128.0,68.0,52.0,32.0,21.0,12.0,8.0,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,PRV51005,Yes,2.088412,0.980747,244.300429,196.533055,6749.216309,6,495,outpatient,1165,4,26,4.500000,116.500000,158.0,0.0,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0,6.0,4.0,809.0,327.0,232.0,186.0,113.0,79.0,61.0,40.0,25.0,1.0,NaN,NaN,NaN,NaN,NaN
4,PRV51007,No,1.958333,0.985303,513.527778,243.681944,5694.444444,10,58,both,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0,5.0,7.0,71.0,32.0,28.0,21.0,14.0,12.0,8.0,6.0,5.0,1.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,0.983401,384.642857,200.629252,8548.428571,1,24,outpatient,28,1,2,5.500000,2.800000,3.0,0.0,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4,1.0,1.0,26.0,11.0,8.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5406,PRV57760,No,1.318182,1.000000,216.818182,216.022727,5520.545455,3,9,outpatient,22,1,3,4.600000,4.400000,7.0,0.0,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0,2.0,2.0,20.0,10.0,8.0,5.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407,PRV57761,No,2.390244,0.935979,229.756098,157.134674,11248.414634,2,67,outpatient,82,1,2,4.500000,8.200000,16.0,0.0,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3,5.0,2.0,81.0,47.0,27.0,23.0,11.0,8.0,4.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5408,PRV57762,No,1.000000,1.000000,1900.000000,1900.000000,19008.000000,1,1,outpatient,1,1,1,5.000000,1.000000,1.0,0.0,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Impute na values with 0

In [317]:
train_data = train_data.replace(np.nan,0)

#### Pickling dataset 2

In [318]:
train_data.to_csv('train_dataset2.csv')

# Feature Engineering 3

In [319]:
train_data3 = pd.read_csv("train_dataset2.csv")

In [320]:
train_data3 = train_data3.drop(columns= ['NumDistincClmDiagCode1','NumDistincClmDiagCode2','NumDistincClmDiagCode3', \
                   'NumDistincClmDiagCode4','NumDistincClmDiagCode5','NumDistincClmDiagCode6', \
                   'NumDistincClmDiagCode7','NumDistincClmDiagCode8','NumDistincClmDiagCode9', \
                   'NumDistincClmDiagCode10','NumDistincClmProCode1','NumDistincClmProCode2','NumDistincClmProCode3', \
                  'NumDistincClmProCode4','NumDistincClmProCode5'], axis=1)

In [321]:
train_data3 = train_data3.rename(columns={'numDiffDiagnosisCode':'NumUniqClmAdmitDiagCOde',
                                        'numDiffGroupDiagCode':'NumUniqGroupDiagCode'})

In [322]:
train_data3

,Unnamed: 0,Provider,PotentialFraud,DaysAdmitted,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,TotalAnnClaimAmt,NumOfDoctors,NumOfPatients,ServiceType,NumOfClms,numStates,numCounties,AvgChronic,AvgClaim,NumUniqClmAdmitDiagCOde,NumUniqGroupDiagCode,AvgAgeWhenServed,NumOfDuplicatedBeneID,NumOfDuplicatedAttendingPhysician,NumOfDuplicatedClaims,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12,Gender1,Gender2,Race1,Race2,Race3,Race5,NumDistincOpPhy,NumDistincOtherPhy
0,0,PRV51001,No,2.440000,0.975656,4399.200000,1061.072000,21582.240000,14,24,both,25,1,7,5.888889,2.777778,11.0,5.0,77.729167,0.0,3.0,11.0,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0,9,15,20,4,0,0,5.0,8.0
1,1,PRV51003,Yes,4.674242,0.912167,5090.575758,1298.615410,11914.909091,44,117,both,132,3,23,5.000000,12.000000,58.0,58.0,68.485755,0.0,3.0,25.0,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0,49,68,94,22,0,1,7.0,22.0
2,2,PRV51004,No,2.429530,0.978485,352.214765,250.363050,7604.483221,38,138,outpatient,149,9,28,4.500000,14.900000,25.0,0.0,71.937802,2.0,16.0,72.0,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0,45,93,113,20,5,0,19.0,26.0
3,3,PRV51005,Yes,2.088412,0.980747,244.300429,196.533055,6749.216309,6,495,outpatient,1165,4,26,4.500000,116.500000,158.0,0.0,69.503649,113.0,6.0,530.0,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0,208,287,382,110,3,0,6.0,4.0
4,4,PRV51007,No,1.958333,0.985303,513.527778,243.681944,5694.444444,10,58,both,72,2,6,4.000000,8.000000,17.0,3.0,67.491379,3.0,3.0,29.0,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0,27,31,47,11,0,0,5.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,5405,PRV57759,No,3.142857,0.983401,384.642857,200.629252,8548.428571,1,24,outpatient,28,1,2,5.500000,2.800000,3.0,0.0,73.916667,3.0,1.0,15.0,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,13,11,19,1,0,4,1.0,1.0
5406,5406,PRV57760,No,1.318182,1.000000,216.818182,216.022727,5520.545455,3,9,outpatient,22,1,3,4.600000,4.400000,7.0,0.0,64.666667,3.0,1.0,13.0,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,3,7,2,0,0,2.0,2.0
5407,5407,PRV57761,No,2.390244,0.935979,229.756098,157.134674,11248.414634,2,67,outpatient,82,1,2,4.500000,8.200000,16.0,0.0,73.800995,3.0,2.0,32.0,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0,33,34,58,5,1,3,5.0,2.0
5408,5408,PRV57762,No,1.000000,1.000000,1900.000000,1900.000000,19008.000000,1,1,outpatient,1,1,1,5.000000,1.000000,1.0,0.0,67.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,0.0,0.0


In [323]:
train_data3.to_csv('train_dataset3.csv')

## 43 features remaining after feature engineering